In [137]:
import os 
import numpy as np 
import os
import re 
import csv 
import math 
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import word_tokenize
import pandas as pd 

In [138]:

docs = {}
labels = {}
#I only work if you run me in the same folder as the actual text files 

with open('training_labels.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            #print (row) #these are in the order idx,id,score
            line_count += 1
        else:
            #print(row)
            id = int(row[1])
            score= float(row[2])
            labels[id]=score
            line_count += 1
    print(line_count)

for root, dirs, files in os.walk("./corpus/"):
    for file in files:
        if file.endswith(".txt"):
            id = re.sub("[^0-9]", "", file)
            id = int(id) 
            path_file = os.path.join(root,file)
            curdir = path_file
            file = open(curdir, 'r') 
            text = file.read()
            
            docs[id] = text

535


In [139]:
train_x,train_y = [],[]
for k in labels:
    try:
        if docs[k] and labels[k]:
            train_x.append(docs[k]) 
            train_y.append(labels[k])
    except KeyError:
        pass

#these two should match 
# print (len(train_x))
# print (len(train_y))


# indices = math.floor(len(train_x)*.2)
# indices = list(range(0,indices))
# test_x = [train_x[x] for x in indices]
# test_y = [train_y[x] for x in indices]
# train_x = train_x[indices[-1]+1:] 
# train_y = train_y[indices[-1]+1:]
# print(len(test_x))
# print(len(train_x))
# print(len(test_x) + len(train_x))

In [140]:
from sklearn.feature_extraction.text import CountVectorizer

def vectorize(txt):
    count_vect = CountVectorizer(stop_words=stopwords.words('english'),max_features = 6,strip_accents ='ascii')
    #print(count_vect.get_feature_names)
    vects = count_vect.fit_transform(txt)
    print(vects.shape) 
    return count_vect

from sklearn.preprocessing import normalize 
def normalize_vect(v):
    return normalize(v)
    
v = vectorize(train_x)
imp_words = v.get_feature_names()
print(imp_words)

(434, 6)
['data', 'information', 'may', 'personal', 'privacy', 'use']


In [141]:
id_ct = {}
for id in docs:
    id_ct[id] = [0]*len(imp_words)
for id in id_ct:
    txt = docs[id] 
    for x in range(len(imp_words)):
        word = imp_words[x]
        if word in txt:
            id_ct[id][x] +=1
            

In [142]:
# for k in id_ct:
#     print(id_ct[k])

In [143]:
dat = pd.DataFrame(data = id_ct)

In [144]:
dat.head()

,10051,10062,10063,10065,10074,10077,10080,10089,10092,10170,...,939,941,943,945,948,952,954,963,965,974
0,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,0,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,0,1,1,0,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1


In [145]:
dat = dat.transpose()
dat.head()

,0,1,2,3,4,5
10051,1,1,1,1,1,1
10062,1,1,1,1,0,1
10063,1,1,1,1,1,1
10065,1,1,1,1,1,1
10074,1,1,1,1,0,1


In [146]:
dat.columns = imp_words

In [147]:
dat.head()

,data,information,may,personal,privacy,use
10051,1,1,1,1,1,1
10062,1,1,1,1,0,1
10063,1,1,1,1,1,1
10065,1,1,1,1,1,1
10074,1,1,1,1,0,1


In [155]:
dat['iD'] = dat.index

In [156]:
dat.head()

,data,information,may,personal,privacy,use,index1,iD
10051,1,1,1,1,1,1,10051,10051
10062,1,1,1,1,0,1,10062,10062
10063,1,1,1,1,1,1,10063,10063
10065,1,1,1,1,1,1,10065,10065
10074,1,1,1,1,0,1,10074,10074


In [ ]:
#add me to the regular dataframe 
data = pd.read_csv("final.csv")
frames = [data,dat]
data =  data.merge(dat,how='left',on='iD')
display(data.head(n=10))